In [69]:
import fasttext

In [70]:
model = fasttext.load_model("cc.ru.300.bin")

In [1]:
import copy
import time
import numpy as np
import pandas as pd

from gensim.models.fasttext import FastText, load_facebook_model
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import word2vec

In [60]:
# pip install --upgrade gensim==4.2.0

In [61]:
pip show gensim

Name: gensimNote: you may need to restart the kernel to use updated packages.
Version: 4.2.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only

Location: c:\users\bimas\anaconda3\lib\site-packages
Requires: scipy, smart-open, numpy, Cython
Required-by: 


# Load data

In [62]:
data = pd.read_csv('proc.zip')
data.dropna(inplace=True)
data.head()

,text_review,review_id,preprocessed_review,lemmas
0,Мне если честно аромат не очень.... Всегда ста...,75b8359b-2336-4759-880d-45c531c3470c,честно аромат очень стараться прислушиваться а...,я если честно аромат не очень всегда стараться...
1,Туалетная вода от «Nina Ricci» - Nina L’Eau.Пр...,e7077664-72ae-49f5-a1b8-7b683fd07728,туалетный вода nina ricci nina приобрести инте...,туалетный вода от nina ricci nina l eau приобр...
2,Очень приятный и свежий аромат. Ярко стартует ...,f99d2450-90dc-4914-9c02-6cf1d41de700,очень приятный свежий аромат ярко стартовать ц...,очень приятный и свежий аромат ярко стартовать...
3,Cтал счастливым обладателем запечатанной миниа...,06fc790e-6dde-4835-ac00-5c1263884033,cталый счастливый обладатель запечатать миниат...,cталый счастливый обладатель запечатать миниат...
4,"Увидев на стенде Летуаль новинку от Shiseido, ...",4abc59a2-ebf0-44ed-b9fe-0ebe71f57807,увидеть стенд летуаля новинка shiseido смочь п...,увидеть на стенд летуаля новинка от shiseido н...


In [66]:
data['text_review'][10]

'Всем привет!Меня зовут Ангелина и я просто обожаю парфюм)На сегодняшний день моя коллекция насчитывает более 50 флаконов и огромное количество отливантов и пробников, которые постоянно пополняются.Проснувшись рано утром, я всегда думаю о том, какой аромат будет сопровождать меня сегодня. Ведь именно парфюм помогает задать тон сегодняшнему дню.Итак, сегодня мой выбор остановился на Armani. Этот дом я люблю, они выпускают много достойных ароматов, которые становятся настоящими бестселлерами. Вашему вниманию- Emporio Armani Because It’s You Giorgio Armani .Этот аромат принадлежит к группе цветочные фруктовые. Он был выпущен в 2017 году.Парфюмер: Anne Flipo.Пирамида:Верхние ноты: Малина, Лимон и Нероли;Средняя нота: Роза;Базовые ноты: Ваниль, Мускус и Amberwood.На мне аромат стартует нежной ароматной малиной с лимончиком. Затем лёгкой дымкой начинает появляться прекрасное томное сердце из роз🌹.База нежная как сама кожа, и ваниль...не кондитерская, немного древесная.Аромат очень объемный и

In [67]:
data['preprocessed_review'][10]

'весь привет звать ангелина просто обожать парфюм сегодняшний день коллекция насчитывать флакон огромный количество отливант пробник который постоянно пополняться проснуться рано утром думать аромат сопровождать сегодня именно парфюм помогать задать сегодняшний день итак сегодня выбор остановиться armani любить выпускать достойный аромат который становиться настоящий бестселлер внимание emporio armani because giorgio armani аромат принадлежать группа цветочный фруктовый выпустить парфюмер anne flipo пирамида верхний нота малина лимон нероли средний нота роза базовый нота ваниль мускус amberwood аромат стартовать нежный ароматный малина лимончик затем гкий дымка начинать появляться прекрасный томный сердце роза база нежный кожа ваниль кондитерский немного древесный аромат очень объёмный летучий длинный шлейф шлейф прекрасный лёгкий игривый because невероятно женственный аромат полный страсть пробуждать потаённый инстинкт абсолют роза мгновенно очаровывать пленять навеки флиртовать собла

### Create corpus

In [5]:
sentence_corpus = [line.split(' ') for line in data['lemmas'].values if len(line) > 10 and len(line) <= 150]
# sentence_corpus[:2]

# Train model

In [6]:
class callback(CallbackAny2Vec):
    '''Callback for Word2vec with resetting loss on the end of each epoch.'''

    def __init__(self):
        self.epoch = 1

        self.epoch = 1
        self.losses = []
        self.cumu_loss = 0.0
        self.previous_epoch_time = time.time()

        self.best_model = None
        self.best_loss = 1e+30

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()

        norms = [np.linalg.norm(v) for v in model.wv.vectors]
        now = time.time()
        epoch_seconds = now - self.previous_epoch_time
        self.previous_epoch_time = now
        self.cumu_loss += float(loss)
        print(f"Loss after epoch {self.epoch}: {loss} (cumulative loss so far: {self.cumu_loss}) "+\
              f"-> epoch took {round(epoch_seconds, 2)} s - vector norms min/avg/max: "+\
              f"{round(float(min(norms)), 2)}, {round(float(sum(norms)/len(norms)), 2)}, {round(float(max(norms)), 2)}")
        self.epoch += 1

        self.losses.append(float(loss))
        
        # reset loss inside model
        model.running_training_loss = 0.0

        if loss < self.best_loss:
            self.best_model = copy.deepcopy(model)
            self.best_loss = loss

        if self.epoch % 5 == 0:
            self.plot(path="w2v_training_loss.png")

    def plot(self, path):
        fig, (ax1) = plt.subplots(ncols=1, figsize=(6, 6))
        ax1.plot(self.losses, label="loss per epoch")
        plt.legend()
        plt.savefig(path)
        plt.close()
        print("Plotted loss.")

In [11]:
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

### load model

In [ ]:
# model = load_facebook_model('cc.ru.300.bin')
model = FastText.load_fasttext_format('wiki.ru.bin')

### train model

In [12]:
print("building vocabulary...")
model.build_vocab(sentence_corpus[:500], update=True)

building vocabulary...


In [13]:
print("training Word2Vec...")
callbacker = callback()
model.train(
        sentence_corpus[:500],
#         epochs=model.iter,
        epochs=20,        
    total_examples=model.corpus_count,
        compute_loss=True,
        callbacks=[callbacker]
    )

training Word2Vec...
Loss after epoch 0: 0.0
Loss after epoch 1: 0.0
Loss after epoch 2: 0.0
Loss after epoch 3: 0.0
Loss after epoch 4: 0.0
Loss after epoch 5: 0.0
Loss after epoch 6: 0.0
Loss after epoch 7: 0.0
Loss after epoch 8: 0.0
Loss after epoch 9: 0.0
Loss after epoch 10: 0.0
Loss after epoch 11: 0.0
Loss after epoch 12: 0.0
Loss after epoch 13: 0.0
Loss after epoch 14: 0.0
Loss after epoch 15: 0.0
Loss after epoch 16: 0.0
Loss after epoch 17: 0.0
Loss after epoch 18: 0.0
Loss after epoch 19: 0.0


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\bimas\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\bimas\AppData\Local\Temp/ipykernel_4996/1605362792.py", line 3, in <module>
    model.train(
  File "C:\Users\bimas\anaconda3\lib\site-packages\gensim\models\word2vec.py", line 1083, in train
    self._clear_post_train()
  File "C:\Users\bimas\anaconda3\lib\site-packages\gensim\models\fasttext.py", line 460, in _clear_post_train
    self.wv.adjust_vectors()  # ensure composite-word vecs reflect latest training
  File "C:\Users\bimas\anaconda3\lib\site-packages\gensim\models\fasttext.py", line 1181, in adjust_vectors
    self.vectors[i] += self.vectors_ngrams[nh]
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\bimas\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 20

TypeError: object of type 'NoneType' has no len()

In [16]:
model.get_latest_training_loss()

0.0

In [74]:
data = pd.read_csv('twitter.zip')
data.head()

,text,label,lemmas
0,"@first_timee хоть я и школота, но поверь, у на...",positive,first_timee хоть я и школотый но поверь у мы т...
1,"Да, все-таки он немного похож на него. Но мой ...",positive,да всё таки он немного похожий на он но мой ма...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive,rt katiacheh: ну ты идиотка) я испугаться за т...
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive,rt digger : кто то в угол сидеть и погибать от...
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive,irina_dyshkant вот что значит страшилка :d но ...


In [75]:
emb = [sg_model.get_sentence_vector(line) for line in data["lemmas"]]

In [76]:
labels = [1 if label == 'positive' else 0 for label in data['label']]

In [77]:
len(labels) == len(emb)

True

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [79]:
train_data, test_data, train_labels, test_labels = train_test_split(emb, labels, test_size=0.3, shuffle=True, stratify=labels)

In [80]:
lr = LogisticRegression(n_jobs=-1)
lr.fit(train_data, train_labels)

LogisticRegression(n_jobs=-1)

In [81]:
from sklearn.metrics import roc_auc_score

In [85]:
lr_pred = lr.decision_function(test_data)
print('Logistic regression ROC-AUC:', roc_auc_score(test_labels, lr_pred))

Logistic regression ROC-AUC: 0.84378918855271


# Дообучение 

In [32]:
!wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt

--2022-05-26 01:15:10--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167631 (164K) [text/plain]
Saving to: 'alice.txt.1'

     0K .......... .......... .......... .......... .......... 30%  444K 0s
    50K .......... .......... .......... .......... .......... 61% 1.65M 0s
   100K .......... .......... .......... .......... .......... 91%  757K 0s
   150K .......... ...                                        100% 5.72M=0.2s

2022-05-26 01:15:11 (777 KB/s) - 'alice.txt.1' saved [167631/167631]



In [36]:
import re
from nltk.tokenize import sent_tokenize, RegexpTokenizer  # токенизаторы из NLTK

In [37]:
with open("alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

# убираем переносы строк, токенизируем текст

text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []

# убираем всю пунктуацию и делим текст на слова по пробелу
for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)
    
print(clean_sents[:2])

[['through', 'the', 'looking-glass', 'by', 'lewis', 'carroll', 'chapter', 'i', 'looking-glass', 'house', 'one', 'thing', 'was', 'certain', 'that', 'the', 'white', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', '', 'it', 'was', 'the', 'black', 'kitten’s', 'fault', 'entirely'], ['for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', 'and', 'bearing', 'it', 'pretty', 'well', 'considering', 'so', 'you', 'see', 'that', 'it', 'couldn’t', 'have', 'had', 'any', 'hand', 'in', 'the', 'mischief']]


In [38]:
fasttext.train_unsupervised(input='')

In [3]:
import fasttext

In [1]:
# sg_model = fasttext.train_unsupervised(input='perfumery.txt', dim=300, lr=0.6, word_ngrams=3, pretrainedVectors='cc.ru.300.vec')

In [57]:
compression_opts = dict(method='zip', archive_name='perfumery.zip')  
tables.to_csv('perfumery.zip', index=False, compression=compression_opts) 

In [74]:
with open("data/processed/perfumery.txt", "w", encoding='UTF-8') as file:
    for line in tables.comment_text.values:
        file.write(line + '\n')

In [ ]:
model = fasttext.train_unsupervised(input='perfumery.txt', dim=300, lr=0.07, word_ngrams=3, pretrainedVectors='cc.ru.300.vec', bucket=300000, lrUpdateRate=1)